# Embedding
![rag_embedding](figures/rag_embedding.png)

- 분할된 텍스트를 벡터 표현(임베딩 벡터)으로 변환한다.
- LangChain은 OpenAI, HuggingFace 등 다양한 임베딩 모델을 지원하며, 동일한 인터페이스로 사용할 수 있다.
- [임베딩모델의 메서드](https://api.python.langchain.com/en/latest/embeddings/langchain_core.embeddings.embeddings.Embeddings.html#langchain_core.embeddings.embeddings.Embeddings)

    - **`embed_documents(texts: List[str])`**
        - 여러 문서를 받아 벡터화(임베딩)한다.
        - Context를 벡터화 할 때 사용한다.
    - **`embed_query(text: str)`**
        - 하나의 문자열(문서)을 받아 벡터화한다.
        - Query를 벡터화 할 때 사용한다.


In [8]:
docs = [
        "나는 고양이와 개 중 반려동물로 개를 키우고 싶습니다.",
        "이 강아지 품종은 진도개 입니다. 국제 표준으로 중대형견으로 분류되며 다리가 길어 체고가 높은 편에 속합니다.",
        "日本の市内バスの運賃は主に距離によって決まり、地域やバス会社によって異なる場合があります",                  # 일본의 시내버스 요금은 주로 거리에 따라 결정되며, 지역 및 버스 회사에 따라 다를 수 있습니다.
        "Bus fares in the United States vary from city to city, but are generally around $2.90 for a regular bus.",  # 미국의 버스 요금은 도시마다 다르지만, 일반적으로 정기 버스의 경우 2.90달러 정도입니다.
        "광역버스 요금은 일반 3000원, 청소는 1800원, 어린이 1500원 입니다.", 
]

In [9]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
################################
# 1) OpenAI 사용
from langchain_openai import OpenAIEmbeddings

e_model_id = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=e_model_id)

In [44]:
################################
# 2) Ollama 사용
from langchain_ollama import OllamaEmbeddings

e_model_id = "bge-m3"
embedding_model = OllamaEmbeddings(model=e_model_id)

In [ ]:
################################
# 3) Huggingface 사용
from langchain_huggingface import HuggingFaceEmbeddings

e_model_id = "intfloat/multilingual-e5-large"
embedding_model = HuggingFaceEmbeddings(model=e_model_id)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

c:\Users\Playdata\anaconda3\envs\lang_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--intfloat--multilingual-e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [45]:
# 문서들을 embedding
embedded_docs = embedding_model.embed_documents(docs)

In [46]:
type(embedded_docs), type(embedded_docs[0])

(list, list)

In [47]:
import numpy as np

np.shape(embedded_docs)
# (5: 문서 개수, 1536: 개별 문서의 embedding 벡터의 차원)

(5, 1024)

In [48]:
def cosine_similarity(v1: np.ndarray|list, v2: np.ndarray|list)->float:
    # v1, v2의 코사인 유사도를 반환하는 함수
    v1 = np.array(v1)
    v2 = np.array(v2)
    return (v1 @ v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))

## 코사인 유사도 (-1 ~ 1 사이 값)
# 1: 매우 유사함
# 0: 관계 없는 것
# -1: 정확하게 반대 의미

In [49]:
query = "당신이 좋아하는 동물은 무엇인가요?"
embedded_query = embedding_model.embed_query(text=query)

np.shape(embedded_query)

(1024,)

In [ ]:
# embedded_query와 embedded_docs 간의 코사인 유사도 계산 (1)
for i, ev in enumerate(embedded_docs):
    print(f"{i+1}. {cosine_similarity(ev, embedded_query)}")

1. 0.36055681135000084
2. 0.26502894656727843
3. 0.04863998198846294
4. -0.009661540805491114
5. 0.06463066481689705


In [ ]:
# embedded_query와 embedded_docs 간의 코사인 유사도 계산 (2)
for i, ev in enumerate(embedded_docs):
    print(f"{i+1}. {cosine_similarity(ev, embedded_query)}")

1. 0.6662511687008726
2. 0.4408926487024179
3. 0.20180528508045684
4. 0.224805009320678
5. 0.26837513265566254


In [ ]:
# embedded_query와 embedded_docs 간의 코사인 유사도 계산 (3)
for i, ev in enumerate(embedded_docs):
    print(f"{i+1}. {cosine_similarity(ev, embedded_query)}")

1. 0.8452872050559566
2. 0.7992489104879116
3. 0.7132940936416443
4. 0.6920042253310208
5. 0.7591988811567014


# 벡터 데이터베이스(Vector Database)
- Embedding 된 문서를 Vector Database(Vector Store)에 저장한다.
- 이후 질문(Query)와 관련된 내용을 유사도를 이용해 검색해 질문과 함께 prompt로 만든다. (Retrieve)

![rag_vector_store](figures/rag_vector_store.png)

## 벡터 데이터베이스란
- 벡터 임베딩을 저장하고 관리하는 데이터베이스를 의미한다.
- 모든 데이터는 적절한 임베딩 모델을 활용하면 임베딩 벡터로 변환할 수 있다. 이렇게 변환된 임베딩 벡터를 벡터 데이터베이스에 저장하면 **임베딩 벡터 간의 거리 계산을 통해 데이터 간 유사도를 검색할 수 있다.**
    - **이미지, 텍스트, 음성 등 비정형 데이터**를 임베딩 모델로 **벡터화한 뒤 데이터베이스에 저장**한다.
    - 벡터 간의 **유사도 계산**을 통해 연관성 있는 데이터나 유사한 데이터를 효과적으로 검색할 수 있다.
    - 좋은 검색 결과를 위해서는 벡터의 품질이 중요하다. 그래서 **임베딩 모델(Embedding Model)을 잘 선택하는 것이 중요**하다.
- 벡터 데이터베이스는 이러한 벡터 간 거리 계산에 특화된 데이터베이스다.

## 주요 특징

- **고차원 벡터 저장**
  -  벡터 데이터베이스는 수백에서 수천 차원에 이르는 벡터 데이터를 효율적으로 저장하고 관리한다. 
  -  전통적인 데이터베이스로는 어려운 고차원 벡터 간 유사도 검색을 효율적으로 수행한다.
- **유사성 기반 검색**
  -  벡터 간의 거리를 측정하여 유사한 데이터를 빠르게 검색할 수 있다. 
  -  일반적으로 사용되는 거리계산기법은 다음과 같다.
     - 코사인 유사도(Cosine Similarity)
     - 유클리드 거리(Euclidean Distance)
     - 맨하탄 거리(Manhattan Distance) 
- 비정형 데이터 처리: 텍스트, 이미지, 오디오 등 다양한 비정형 데이터를 벡터로 변환하여 저장하고, 이러한 데이터를 효과적으로 검색할 수 있다.

## 벡터 데이터베이스와 딥러닝
- 벡터 데이터베이스는 딥러닝 기술의 발전과 깊은 관련이 있다.
- 딥러닝 모델은 학습 과정에서 데이터의 특징을 추출하는 방법을 함께 학습한다. 충분한 데이터를 학습한 딥러닝 모델은 **데이터의 특성을 설명하는 특성 벡터(feature vector)를 효과적으로 생성**할 수 있다.
- 이때 추출된 특성 벡터는 고차원 데이터(RAW Data)를 저차원 공간에서 표현한 **임베딩 벡터**다.
    - > **임베딩**은 고차원 데이터를 저차원 공간으로 변환하여 표현하는 방법으로, 정보 손실을 최소화하면서 데이터 간의 의미 있는 관계를 벡터 공간에서 유지한다.
- 딥러닝 모델로 추출한 데이터의 특징(feature vector)을 임베딩 공간에 배치하면, 비슷한 데이터는 가까이, 그렇지 않은 데이터는 멀리 배치된다.
- 이러한 특성을 활용하면 임베딩 벡터 간의 거리를 계산해 유사한 데이터를 효과적으로 검색할 수 있다. 벡터 데이터베이스는 이러한 임베딩 벡터의 특성을 기반으로 개발되었다.
- 딥러닝 기술의 발전과 폭넓은 활용으로 임베딩 데이터의 사용이 증가하면서, 이를 저장하고 관리하는 기능에 특화된 데이터베이스에 대한 수요도 증가해 다양한 벡터 데이터베이스가 등장했다.

## 벡터 데이터베이스의 주요 기능
1. **저장**  
   - 이미지, 텍스트, 음성 등 **비정형 데이터**를 임베딩 모델을 통해 벡터로 변환한 뒤 벡터 데이터베이스에 저장한다.
2. **검색**  
   - 검색하려는 데이터를 임베딩 모델로 변환한 뒤, 벡터 데이터베이스에서 **유사도를 기반**으로 검색한다.
3. **결과 반환**  
   - 벡터 데이터베이스는 저장된 벡터 중 검색 쿼리 임베딩과 가장 가까운 벡터를 찾아 반환한다.

## LLM과 벡터 데이터베이스
- ChatGPT(LLM)의 등장 이후 벡터 데이터베이스는 폭발적인 주목을 받았다.
- 임베딩 벡터의 유사도를 기반으로 문서를 검색하는 RAG(Relevant Augmented Generation) 기술은 LLM의 환각(할루시네이션) 현상을 줄이고, LLM을 추가 학습하지 않고도 최신 정보를 효율적으로 활용할 수 있는 핵심 기법으로 자리 잡았다.
   


## 벡터 데이터베이스 종류
![img](figures/vector_database.png)

<<https://blog.det.life/why-you-shouldnt-invest-in-vector-databases-c0cd3f59d23c>>

### 주요 벡터 데이터베이스 종류
- **Pinecone**
    - 클라우드 기반의 완전 관리형 벡터 데이터베이스 서비스로, 간단한 API를 통해 벡터 데이터를 관리할 수 있다.  
    - 자동 확장성과 고가용성을 제공하며, 실시간 데이터 수집과 유사성 검색에 최적화되어 있다.
    - 가장 쉽게 시작할 수 있는 관리형 서비스를 제공한다.
- **Chroma**
    - 벡터 임베딩을 효율적으로 저장하고 검색할 수 있는 오픈소스 데이터베이스로, AI 및 머신러닝 애플리케이션에 최적화되어 있다.
    - 대규모 임베딩 저장에 최적화되어 있다.
- **FAISS**
    - Facebook AI에서 개발한 고성능 벡터 검색 라이브러리로, 고차원 벡터의 효율적인 유사성 검색을 위해 최적화되어 있다.
    - GPU를 활용해 계산 성능을 높이며, 벡터 양자화 기술을 활용하여 메모리 사용을 최적화한다.
    - 근사 최근접 이웃 검색(ANNS)에 최적화되어 있다.
- **Milvus**
    - 오픈소스 벡터 데이터베이스로, 대규모 벡터 데이터를 효율적으로 저장하고 검색할 수 있다.  
    - 분산 아키텍처를 채택하여 확장성이 뛰어나며, IVF_PQ, DiskANN 등 다양한 인덱싱 알고리즘을 지원한다.
    - 대규모 데이터셋 처리에 가장 적합한 솔루션이다.
- **Weaviate**
    - 오픈소스 벡터 데이터베이스로, 텍스트, 이미지, 오디오 등 다양한 비정형 데이터를 벡터로 저장하고 검색할 수 있다.  
    - GraphQL API를 통해 접근 가능하며, 내장된 머신러닝 모듈을 통해 가장 강력한 의미론적 검색 기능을 제공한다.
- **Qdrant**
    - Rust로 개발된 고성능 벡터 검색 엔진으로, 실시간 근사 최근접 이웃 검색을 제공한다.  
    - 추천 시스템에 특화되어 있으며, 벡터 임베딩 저장과 유사도 쿼리를 효율적으로 수행한다.
- **Elasticsearch**
    - HNSW 알고리즘을 사용하여 벡터 검색을 구현하는 검색 엔진이다.
    - 전통적인 검색 기능과 벡터 검색을 효과적으로 결합할 수 있어, 하이브리드 검색에 가장 적합하다.
- **PGVector**
    - PostgreSQL의 확장 모듈로, 벡터 데이터를 저장하고 유사성 검색을 수행할 수 있게 해준다.  
    - SQL과 통합된 벡터 연산이 가능하며, L2 거리, 코사인 거리, 내적 등 다양한 거리 측정 방식을 지원한다.


# Langchain - Vector Store 연동 
- Langchain은 다양한 벡터 데이터베이스와 연동할 수 있다.
- 벡터 데이터베이스 마다 API가 다르기 때문에, Langchain을 사용하면 동일한 interface로 사용할 수 있다.

## **VectorStore**
- Langchain이 지원하는 모든 벡터 데이터베이스는 **VectorStore** 인터페이스를 구현한다.
- 그래서 Langchain에서는 벡터 데이터베이스를 **Vector Store** 라고 한다.
- https://python.langchain.com/docs/integrations/vectorstores/

### Vector Store 연결
- Vector DB와 연결하는 메소드
- `VectorStore.from_document()`
  - Document들을 insert 하면서 연결.
  - Database가 있으면 연결, 없으면 생성하면서 연결한다.
  - Parameter
    - documents: insert할 문서들을 list[Document]로 전달.
    - embedding model
    - vector db에 연결하기 위한 설정들을 넣어준다.
-`VectorStore()`
  - vector db와 연결만 한다.
  - Database가 있으면 연결, 없으면 생성하면서 연결한다.
  - Parameter
    - embedding model
    - vector db에 연결하기 위한 설정들을 넣어준다.
## InMemoryVectorStore
- langchain에서 제공하는 메모리 기반 벡터 데이터베이스이다.
- Data들을 Dictionary를 사용해 메모리에 저장하며, 검색 할 때 코사인 유사도(cosine similarity)를 계산하여 조회한다.

In [51]:
from dotenv import load_dotenv
load_dotenv()

True

In [52]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
# VectorStore 생성 시 Embedding 모델을 넣어 생성한다.

# DB 연결
vector_store = InMemoryVectorStore(embedding=embedding_model)

In [55]:
# 문서 정의
from langchain_core.documents import Document
d1 = Document(
    page_content="Apple, Pear, Watermelon", # 문서 내용
    id="1", # 문서 ID (식별자)
    metadata={"category": "fruit"} # 문서 정보
)
d2 = Document(
    page_content="Python, C++, Java, C#, Rust", # 문서 내용
    id="2", # 문서 ID (식별자)
    metadata={"category": "IT"} # 문서 정보
)
d3 = Document(
    page_content="Football, Baseball, Basketball", # 문서 내용
    id="3", # 문서 ID (식별자)
    metadata={"category": "sports"} # 문서 정보
)
docs = [d1, d2, d3]

# VectorDB에 저장
vector_store.add_documents(documents=docs)

['1', '2', '3']

In [56]:
# DB와 연결하면서 Document들을 insert(upsert)
vector_store2 = InMemoryVectorStore.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [62]:
# 검색 (query와 유사한 문서를 Vector Store에서 찾는다.)
# query = "SQL"
query = "야구"

result = vector_store.similarity_search_with_score(query=query, k=2)

In [63]:
result

[(Document(id='3', metadata={'category': 'sports'}, page_content='Football, Baseball, Basketball'),
  0.5147806103942447),
 (Document(id='2', metadata={'category': 'IT'}, page_content='Python, C++, Java, C#, Rust'),
  0.15766624806715804)]

# 실습
- `data/olympic.txt` 데이터 사용

1. text loading
2. text split
3. embedding + vector store(InMemoryVectorStore)에 저장
4. query(질의)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
# 1. 문서 loading + 2. Split(Chunking)
loader = TextLoader("data/olympic.txt", encoding="utf-8")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50
)
docs = loader.load_and_split(splitter)
# raw_docs = loader.load() -> docs = splitter.split_documents(raw_docs)
print(len(docs))

61


In [4]:
docs[0]

Document(metadata={'source': 'data/olympic.txt'}, page_content='올림픽')

In [5]:
# 3. embedding + 4. VectorStore에 저장
# VectorDB와 연결
vector_store = InMemoryVectorStore(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")  #임베딩모델
)
# 저장(embedding 처리후 저장 -> vector store가 임베딩 작업은 처리한다.)
vector_store.add_documents(docs)

['b92635c0-d2ae-4185-a184-4c310e226573',
 '70d0a6e4-0367-494b-b7d9-12df943b2513',
 'd23476ee-92d9-414d-ae5e-afcb2a2f5475',
 '9b17cfd5-603e-4aca-9193-3422bf621ed0',
 '93e37224-be82-4333-9c22-7707c6466e93',
 'd1750e1a-f154-4750-9f0f-ca36a7d35f66',
 '4d6311fb-c983-405e-a19c-9c18675e91f3',
 '2c19d36c-b82e-4ad5-8f8e-106d1b2f5b91',
 '88bdb2c4-fd20-4032-a53a-3d3cf55730ea',
 '848af5af-2f96-4244-b7bb-303f029cf335',
 '1bad6756-50bc-48d3-9796-f5feecf1ccc1',
 '0b785787-5c28-4d45-8e73-77d86b8d6f81',
 'a713cebf-892b-4470-80af-0f1924764404',
 'e0084db3-a927-4483-98f2-2bc50512527c',
 '0aa75bd5-b327-4194-9745-12e864b5c415',
 '38f49e23-414d-43fa-a659-4896dce87c98',
 '5b2e447f-a533-47f0-a462-4bd1a84e6c28',
 '7ec9b680-ebe6-46e9-a63e-45d841048b6a',
 '86b81b99-4e07-441c-9a16-6893a317deaa',
 '90d43bc0-29ea-4361-b1cf-28677a00a753',
 '75a95f0c-b4b2-4b84-9aca-044bb98c6681',
 '68cbbeda-43a8-426a-9b55-0886fc684969',
 '479b92fd-cce2-4eaf-beba-c525847b4bb4',
 '195a1cf8-5574-4070-b4e6-8ae372fed83c',
 '0de16793-acd7-

In [ ]:
# 연결 + 저장
# v2 = InMemoryVectorStore.from_documents(
#     embedding=OpenAIEmbeddings("text-embedding-3-small"),
#     documents=docs
# )

In [6]:
# 질문 -> 의미적 유사도가 높은 k(5)개의 문서를 반환.
query = "동계 올림픽에 대해 설명해주세요."    #input("질문:")
results = vector_store.similarity_search_with_score(
    query=query, 
    k=5
)
for result in results:
    print(result[1], result[0].page_content[:200])

0.5847532901520429 하계올림픽
0.5167171204610446 동계올림픽
동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만들고 싶어 했고, 로잔에서 열린 1921년 올림픽 의회에서 겨울판 올림픽을 열기
0.48757257538324067 올림픽
0.47471668364853675 오늘날의 올림픽
1896년 대회때는 14개국에서 241명의 선수단이 참가했지만 2008년 하계 올림픽때는 204개국에서 10,500명의 선수가 참가하는 등 세계적인 대회로 변모했다. 동계 올림픽의 규모는 하계 올림픽 규모보다 작다. 예를 들면 2006 토리노 동계 대회때는 80개국에서 2,508명의 선수가 참가했으며 82개 세부종목이 있었고, 2008 베이
0.44939052012829395 고대올림픽


## MMR(최대 한계 관련성-Maximal Marginal Relevance) 알고리즘 적용
최대 한계 관련성(Maximal Marginal Relevance, MMR) 알고리즘은 정보 검색 및 요약에서 검색 결과의 **관련성**과 **다양성**을 동시에 고려하여 최적의 결과를 제공하는 방법이다. 
이 알고리즘은 사용자 쿼리와의 관련성을 최대화하면서도 중복 정보를 최소화하여 다양한 정보를 제공하는 것을 목표로 한다.

1. **관련성과 다양성의 균형 조절**: MMR은 사용자 쿼리와 문서 간의 유사성 점수와 이미 선택된 문서들과의 다양성 점수를 조합하여 각 문서의 최종 점수를 계산한다. 이를 통해 관련성이 높으면서도 중복되지 않는 문서를 선택한다.

2. **수학적 정의**
   $$
   \text{MMR} = \lambda \cdot \text{Sim}(d, Q) - (1 - \lambda) \cdot \max_{d' \in D'} \text{Sim}(d, d')
   $$

   - $\text{Sim}(d, Q)$: 문서 $d$와 쿼리 $\text{Q}$ 사이의 유사성. (문서 유사성 계산)
   - $\max_{d' \in D'} \text{Sim}(d, d')$: 문서 $d$와 이미 선택된 문서 집합 $D'$ 중 가장 유사한 문서와의 유사성. (문서 다양성 계산)
   - $\lambda$: 유사성과 다양성의 중요도를 조절하는 매개변수(parameter)
3. **적용 분야**: MMR은 정보 검색, 추천 시스템, 문서 요약 등에서 활용된다. 특히 LLM 검색에서 성능 향상이 입증되었다.

### `vectorStore.max_marginal_relevance_search()` 메소드
  - MMR 알고리즘을 적용한 검색을 수행한다.
  - **파라미터**
    - **query**: 사용자로부터 입력받은 검색 쿼리
    - **k**: 최종적으로 선택할 문서의 수
    - **fetch\_k**: MMR 알고리즘 적용 시 고려할 상위 문서의 수
    - **lambda_mult**: 쿼리와의 유사성과 선택된 문서 간의 다양성 사이의 균형을 조절하는 매개변수. $\lambda = 1$이면 유사성만 고려하고, $\lambda = 0$이면 다양성만을 최대화한다.
    - **filter**: 검색 결과를 필터링할 조건을 지정한다.


In [7]:
query = "동계 올림픽에 대해 설명해줘."
mmr_result = vector_store.max_marginal_relevance_search(
    query=query,
    k=5, #  최종 결과 문서 개수
    fetch_k=20, # 처음 검색할 문서개수.
    lambda_mult=0.5   # 1에 가까울수록 유사성에 0에 가까울 수록 다양성을 최대화.
)

for result in mmr_result:
    print(result.page_content[:200])
    print("------------------")

하계올림픽
------------------
- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.
- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원
------------------
고대 올림피아 경기가 처음 열린 시점은 보통 기원전 776년으로 인정되고 있는데, 이 연대는 그리스 올림피아에서 발견된 비문에 근거를 둔 것이다. 이 비문의 내용은 달리기 경주 승자 목록이며 기원전 776년부터 4년 이후 올림피아 경기 마다의 기록이 남겨져 있다. 고대 올림픽의 종목으로는 육상, 5종 경기(원반던지기, 창던지기, 달리기, 레슬링, 멀리뛰기)
------------------
동계올림픽
동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만들고 싶어 했고, 로잔에서 열린 1921년 올림픽 의회에서 겨울판 올림픽을 열기
------------------
캐나다에서 열리는 두 번째 동계 올림픽이고, 동/하계 올림픽을 합쳐 캐나다에서 3번째로 개최되는 올림픽이다.
------------------
